In [1]:
# Viterbi Algorithm(隐序列推测)
# O(Obvious-line):      the Obvious-line we got at last step 
# LM(Launch Matrix):    a 2*2 matrix to save launch probability
# CM(Change Matrix):    a 2*2 matrix to save change probability

import numpy as np

def Viterbi(O,LM,CM):
    LM = np.log(LM)
    CM = np.log(CM)
    pi=[[1,1]]
    for i in range (1,len(O)):
        pi.append([0,0])
        if O[i] == "R":
            pi[i][0] = LM[0][0]+max(pi[i-1][0]+CM[0][0],pi[i-1][1]+CM[1][0])
            pi[i][1] = LM[1][0]+max(pi[i-1][0]+CM[0][1],pi[i-1][1]+CM[1][1])
        else:
            pi[i][0] = LM[0][1]+max(pi[i-1][0]+CM[0][0],pi[i-1][1]+CM[1][0])
            pi[i][1] = LM[1][1]+max(pi[i-1][0]+CM[0][1],pi[i-1][1]+CM[1][1])
    
    if pi[len(O)-1][0] >= pi[len(O)-1][1]:
        H = "U";pr_H = "U"
    else:
        H = "D";pr_H = "D"
    
    for i in range (len(O)-2,-1,-1):
        if pr_H == "D":
            if pi[i][0]+CM[0][1] > pi[i][1]+CM[1][1]:
                pr_H="U"
            else:
                pr_H="D"
        if pr_H=="U":
            if pi[i][0]+CM[0][0] > pi[i][1]+CM[1][0]:
                pr_H="U"
            else:
                pr_H="D"
        H=pr_H+H
        
    return H

In [20]:
import csv

def F(x):
    if x == 'U':
        return 0
    else:
        return 1
    
def f(x):
    if x == 'R':
        return 0
    else:
        return 1
def right(O,Hdline,days,x):
    t_num=0
    f_num=0
    for i in range(len(O)-days-x*50,len(O)-x*50):
        if f(O[i]) == F(Hdline[i]):
            t_num=t_num+1
        else:
            f_num=f_num+1
    return float(t_num)/float(f_num+t_num)

def main(filename,x):
    #print("%-15s %-15s %-15s %-15s"%("偏移天数","投资收益","实际涨幅","准确率"))
    Data = []
    with open(filename,'rb') as ff:
        reader = csv.reader(ff)
        for row in reader:
            Data.append(row)
    O = ""
    Price = []
    Rate0 = []
    for i in range(len(Data)-2):
        if(Data[i+2][4]!="null" and Data[i+1][4]!="null"):
            Price.append(Data[i+1][4])
            Rate0.append(float(Data[i+2][4])/float(Data[i+1][4]))
            if (float(Data[i+2][4])-float(Data[i+1][4])>=0):
                O = O + "R"
            else:
                O = O + "G"
    Hdline = Viterbi(O,[[0.6, 0.4], [0.4, 0.6]],[[0.6, 0.4], [0.4, 0.6]])

    Price_my = 1
    flag = False
    days = 300
    for i in range (len(Price)-days-x*50,len(Price)-x*50):
        if flag == False:
            Price_my = Price_my*1.0
        else:
            Price_my = Price_my*Rate0[i]
        if Hdline[i-1] == "U" and  Hdline[i] == "D":
            flag = False
        elif Hdline[i-1] == "D" and Hdline[i] == "U":
            flag = True
    print("%-11.6s %-11.2f %-11.2f %-11.2f"%(x*50,round(Price_my,2),
                                             float(Price[len(Price)-1-x*50])/float(Price[len(Price)-1-days-x*50]),right(O,Hdline,days,x)))

In [22]:
print("%-15s %-15s %-15s %-15s"%("偏移天数","投资收益","实际涨幅","准确率"))
for i in range(100):
    main('600000.csv',i)

偏移天数    投资收益    实际涨幅    准确率      
0           1.39        0.91        0.77       
50          1.43        1.03        0.77       
100         1.48        1.05        0.78       
150         1.54        1.03        0.81       
200         1.53        1.02        0.81       
250         1.42        0.96        0.80       
300         1.57        1.01        0.79       
350         2.02        1.24        0.81       
400         2.13        0.97        0.81       
450         2.22        1.11        0.82       
500         2.98        1.18        0.81       
550         4.18        1.81        0.82       
600         4.28        1.89        0.83       
650         3.24        1.55        0.79       
700         3.21        2.05        0.80       
750         2.63        1.72        0.78       
800         2.36        1.52        0.79       
850         2.17        1.22        0.76       
900         2.13        0.93        0.77       
950         2.27        0.85        0.77       
1000  